In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
  !pip install datasets transformers sacrebleu torch sentencepiece transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [3]:
cd '/content/drive/MyDrive/Capstone'

/content/drive/MyDrive/Capstone


In [4]:
pwd

'/content/drive/MyDrive/Capstone'

In [5]:
import warnings 
warnings.filterwarnings('ignore')
import numpy as np
from transformers import pipeline
from transformers import T5ForConditionalGeneration, T5Tokenizer
import datasets
import random
import pandas as pd
from IPython.display import display, HTML
import torch
from datasets import load_dataset, load_metric



In [6]:
summarizer = pipeline("summarization")


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [7]:
mod_name1= 'Kaggle_finetuned_model'
mod_name2 = 'WMT_finetuned_model'
mod_name3 = 'Subtitle_finetuned_model'
tokenizer1 = T5Tokenizer.from_pretrained(mod_name1)
tokenizer2 = T5Tokenizer.from_pretrained(mod_name2)
tokenizer3 = T5Tokenizer.from_pretrained(mod_name3)
model1= T5ForConditionalGeneration.from_pretrained(mod_name1)
model2 = T5ForConditionalGeneration.from_pretrained(mod_name2)
model3 = T5ForConditionalGeneration.from_pretrained(mod_name2)

In [8]:
raw_datasets = load_dataset("TenzinKhorloVIT/wmt_14_en_fr")
metric = load_metric("sacrebleu")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset wmt_14_en_fr downloaded and prepared to /root/.cache/huggingface/datasets/TenzinKhorloVIT___wmt_14_en_fr/en-fr/1.0.0/f24a05e7c9e71017a1e2c4eb11aef6cde966251a4de6b116e67256dcd2e0453c. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [9]:

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
show_random_elements(raw_datasets["train"])

,translation
0,"{'en': 'Another important element will be the Stockholm Follow-up Group.', 'fr': 'Un autre élément important sera le groupe de suivi de Stockholm.'}"
1,"{'en': 'I have drawn on many ideas from my fellow delegates, but also from many institutions in the sphere I mentioned above.', 'fr': 'J' ai exploité avec reconnaissance les idées de mes collègues, ainsi que celles de nombreuses institutions dans le domaine dont je viens de parler.'}"
2,"{'en': 'I hope that in the light of what President Prodi and Commissioner Verheugen have just had to say, the record will have been put straight by the end of this debate.', 'fr': 'Après ce que nous venons d'entendre du président de la Commission, M. Prodi, et de M. le commissaire Verheugen, j'espère que les choses redeviendront claires au terme de ce débat.'}"
3,"{'en': 'So let me at this early stage express my best wishes for the success of the French Presidency, the President and Government of France, and let me also say that there is not just a desire for success, there is a need for success, a great success.', 'fr': 'Et je vais, d'ores et déjà, souhaiter, et pas seulement souhaiter, mais dire aussi que nous avons besoin d'un succès, d'un grand succès de la présidence française, de son Président et de son gouvernement.'}"
4,"{'en': 'We warned the British Labour Government months ago about the dangers of an open-ended commitment of British troops to Sierra Leone.', 'fr': 'Voilà plusieurs mois, nous avions mis en garde le gouvernement travailliste des dangers d'un engagement ouvert des troupes britanniques en Sierra Leone.'}"


In [10]:
def ensemble_translate(text):
    # Tokenize the input text
    input_ids1 = tokenizer1.encode(text, return_tensors='pt')
    input_ids2 = tokenizer2.encode(text, return_tensors='pt')
    input_ids3 = tokenizer3.encode(text, return_tensors='pt')

    # Generate translations from each model
    with torch.no_grad():
        translation1 = model1.generate(input_ids1)
        translation2 = model2.generate(input_ids2)
        translation3 = model3.generate(input_ids3)

    # Decode the generated translations
    translation1_text = tokenizer1.decode(translation1[0], skip_special_tokens=True)
    translation2_text = tokenizer2.decode(translation2[0], skip_special_tokens=True)
    translation3_text = tokenizer3.decode(translation3[0], skip_special_tokens=True)

    # Combine the translations using a simple voting mechanism
    translations = [translation1_text, translation2_text, translation3_text]
    final_translation = max(set(translations), key=translations.count)

    return final_translation

In [11]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu

def calculate_bleu_score(reference_sentences, generated_sentences):
    """
    Calculate the BLEU score for a model's generated sentences.

    Args:
        reference_sentences (list): List of reference sentences.
        generated_sentences (list): List of generated sentences.

    Returns:
        float: The BLEU score.
    """
    references = [[ref.split()] for ref in reference_sentences]
    hypotheses = [sent.split() for sent in generated_sentences]

    # Calculate the BLEU score
    bleu_score = corpus_bleu(references, hypotheses)

    return bleu_score


In [12]:
input_data = raw_datasets['train']['translation']

In [14]:
inp_text = []
output =[]
for i in range(len(input_data)):
  inp_text.append(input_data[i]['en'])
  output.append(input_data[i]['fr'])

In [15]:
model_translation = [ensemble_translate(text) for text in inp_text]

In [1]:
bleu_score = calculate_bleu_score(output, model_translation)

In [4]:
print("Bleu Score of the ensemble model: {}".format(bleu_score*100))

Bleu Score of the ensemble model: 14.245
